<a href="https://colab.research.google.com/github/esequielsantos/ediBot/blob/main/EdiBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalar Gemini

---



In [ ]:
!pip install -q -U google-generativeai

## Set up my API key

To use the Gemini API, you'll need an API key. If you don't already have one, create a key in Google AI Studio.

<a class="button" href="https://aistudio.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`. Then pass the key to the SDK:


In [ ]:
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Listar modelos disponiveis


In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


Configurando o modelo
Temperaura:

In [ ]:
gen_config = {
    "candidate_count": 1,
    "temperature": 0.5,
}

Segurança:

In [ ]:
saf_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE",
}

Usar **modelo** escolhido acima

In [ ]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro-latest', safety_settings= saf_settings, generation_config= gen_config)

Enviando mensagem ao modelo

In [ ]:
response = model.generate_content("Estou programndo em Phyton. Sugira 5 opções indispensáveis de conhecermos no aprendizado de Phyton")
#print(model.generate_content("Estou programndo em Phyton. Sugira 5 opções indispensáveis de conhecermos no aprendizado de Phyton").text)

Exibindo Resposta

In [ ]:
print(response.text)

5 Opções Indispensáveis para o Aprendizado de Python:

1. **Listas de Compreensão:** Uma sintaxe concisa para criar novas listas a partir de elementos existentes, filtrando e transformando-os.
2. **Dicionários:** Estruturas de dados que mapeiam chaves para valores, permitindo acesso rápido e fácil a dados com base em chaves.
3. **Funções Lambda:** Funções anônimas de uma linha que podem ser usadas como argumentos para outras funções ou armazenadas em variáveis.
4. **Geradores:** Objetos iteráveis que produzem valores sob demanda, economizando memória e melhorando a eficiência.
5. **Módulos:** Arquivos Python que contêm funções, classes e variáveis que podem ser importados para reutilização em outros programas.




> Adicionar aspas INICIANDO O EdiBot

**Iniciando**



In [ ]:
chat = model.start_chat(history=[])

In [ ]:
from datetime import datetime

In [ ]:
def edibot(prompt, history):
  if not chat.history:
    # Obter o horário atual
    agora = datetime.now()

    # Extrair a hora do horário atual
    hora = agora.hour

    # Definir a variável do período do dia
    periodo_dia = ""

    # Verificar o período do dia
    if 6 <= hora < 12:
      periodo_dia = "manhã"
    elif 12 <= hora < 18:
      periodo_dia = "tarde"
    else:
      periodo_dia = "noite"

    bemvindo = model.generate_content(f"Você é um atendente de uma empresa de informatica chamada ByteMe chamado EdiBot. Escreva uma mensagem de saudação de {periodo_dia} e pergunte como pode ajudar").text
    return bemvindo
  else:
    prompt = prompt.lower()
    if prompt == "history":
        return chat.history
    elif 'obrigado' | 'tchau' | 'fim' in prompt:
      agradecer = model.generate_content("Você é um atendente de uma empresa de informatica chamada ByteMe. Escreva uma mensagem curta de agradecimento por ter entrado em contato").text
      return agradecer
    elif prompt == "next":
      continuar = model.generate_content("Escreva uma mensagem curta e agradavél perguntando se deseja mais alguma informação").text
      return continuar
    else:
      response = chat.send_message(prompt)
      return response.text

**EdiBot:** Boa tarde! Obrigado por entrar em contato com a ByteMe. Meu nome é EdiBot. Como posso ajudá-lo hoje? 

quero saber mais sobre o Ryzen 7

 -  **AMD Ryzen 7**

O AMD Ryzen 7 é uma série de processadores de alto desempenho para desktops e laptops. Eles são projetados para oferecer uma combinação de velocidade, eficiência e recursos avançados para uma ampla gama de usuários, incluindo criadores de conteúdo, jogadores e profissionais.

**Características do Ryzen 7:**

* **Arquitetura Zen:** Baseado na avançada arquitetura Zen da AMD, que oferece alta eficiência e desempenho por watt.
* **Vários núcleos e threads:** Os processadores Ryzen 7 possuem um alto número de núcleos e threads, permitindo que eles executem várias tarefas simultaneamente com facilidade.
* **Cache grande:** Eles possuem um grande cache L3, o que ajuda a reduzir a latência e melhorar o desempenho geral.
* **Suporte a PCIe 4.0:** Os processadores Ryzen 7 suportam a interface PCIe 4.0 de alta velocidade, permit

Subir para o GitHub